In [1]:
from latentneural.lorenz import LorenzGenerator, ic
from latentneural.data import DataManager
from latentneural.utils import logger
import numpy as np
import tensorflow as tf
import os
import yaml
from copy import deepcopy

In [2]:
step = 0.01 # Time step in the simulatio (in seconds)
stop = 1 # Final time for the simulation (in seconds)
neurons = 30 # Number of neurons
base_rate = 5 # Base frequency (in Hz)
latent_dim = 3 # Lorenz latent variables encoded in neural activity (relevant+irrelevant)
relevant_dim = 2 # Lorenz latent variables encoded in behavioural activity (relevant)
behaviour_dim = 4 # Behavioural channels
conditions = 1 # Number of conditions to be tested (each condition has different weight matrices to convert latent variables into firing rates and behaviour)
trials = 1000 # Number of trials to be tested for each condition
initial_conditions = ic.uniform(low=-10, high=10) # Sample initial conditions of Lorenz system from uniform distribution
behaviour_sigma=1 # Standard deviation noise added to behaviour
seed=0

In [3]:
with open(os.path.join('..', 'latentneural', 'data', 'storage', \
                       'lorenz', 'grid', 'settings_template.yaml'), 'r') as f:
   training_settings_template = yaml.safe_load(f)
print(training_settings_template)

{'data': {'behavioural_keys': {'test': 'test_behaviours', 'train': 'train_behaviours', 'validation': 'valid_behaviours'}, 'dataset_filename': 'dataset.h5', 'directory': './latentneural/data/storage/lorenz/grid/{{name}}', 'latent_keys': {'test': 'test_latent', 'train': 'train_latent', 'validation': 'valid_latent'}, 'metadata_filename': 'metadata.json', 'neural_keys': {'test': 'test_data', 'train': 'train_data', 'validation': 'valid_data'}}, 'model': {'settings': {'default_layer_settings': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 0.1}, 'type': 'l2'}}, 'encoded_dim': 64, 'layers': {'encoder': {'dropout': 0.05, 'var_max': 0.1, 'var_min': 0.1}, 'decoder': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 1}, 'type': 'l2'}, 'original_cell': False, 're

In [4]:
np.random.seed(seed)
tf.random.set_seed(seed)

for behaviour_sigma in [0.5, 1, 2]:
    for base_rate in [5, 10, 15]:
        for train_num in [50, 100, 200]:
            inner_seed = np.random.randint(0, 2**32 - 1)
            g: LorenzGenerator = LorenzGenerator()
            time, behaviour, spikes, rates, behaviour_w, neural_w, latent = \
                g.generate_spikes_and_behaviour(
                    step=step,
                    stop=stop,
                    n=neurons,
                    base=base_rate,
                    l=latent_dim,
                    b=relevant_dim,
                    y=behaviour_dim,
                    behaviour_sigma=behaviour_sigma,
                    trials=trials + 2 * train_num,
                    initial_conditions=initial_conditions,
                    conditions=1,
                    seed=inner_seed
                )

            c_i = 0
            generation_settings = dict(
                step=step,
                stop=stop,
                neurons=neurons,
                base_rate=base_rate,
                latent_dim=latent_dim,
                relevant_dim=relevant_dim,
                behaviour_dim=behaviour_dim,
                conditions=conditions,
                behaviour_sigma=behaviour_sigma,
                trials=trials + 2 * train_num,
                initial_conditions={'type': 'uniform', 'arguments': {'min': -10, 'max': 10}},
                selected_condition=c_i,
                seed=inner_seed)

            valid_num = train_num
            test_num = trials

            dataset, settings = DataManager.build_dataset(
                neural_data=spikes[c_i,:,:,:],
                behaviour_data=behaviour[c_i,:,:,:],
                settings=generation_settings,
                noisless_behaviour_data=latent[c_i,:,:,-behaviour_w.shape[1]:] @ behaviour_w[c_i,:,:],
                rates_data=rates[c_i,:,:,:],
                latent_data=latent[c_i,:,:,:],
                time_data=time,
                behaviour_weights=behaviour_w[c_i,:,:],
                neural_weights=neural_w[c_i,:,:],
                train_pct=train_num,
                valid_pct=valid_num,
                test_pct=test_num
            )
            dirname = 'train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma)
            data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', 'grid', \
                                    dirname)
            DataManager.store_dataset(
                dataset=dataset,
                settings=settings,
                directory=data_dir)
            logger.info('Dataset stored.')
            
            training_settings = deepcopy(training_settings_template)
            training_settings['data']['directory'] = \
                training_settings['data']['directory'].replace('{{name}}', dirname)
            training_settings['output']['directory'] = \
                training_settings['output']['directory'].replace('{{name}}', dirname)
            # keeping increase per epoch constant (updates are based on batches)
            training_settings['runtime']['weights']['update_rate'][1] = \
                training_settings['runtime']['weights']['update_rate'][1] * 100 / train_num
            training_settings['runtime']['weights']['update_rate'][2] = \
                training_settings['runtime']['weights']['update_rate'][2] * 100 / train_num
            # keeping start epoch constant (start time is based on batches)
            training_settings['runtime']['weights']['update_start'][1] = \
                training_settings['runtime']['weights']['update_start'][1] * train_num / 100
            
            with open(os.path.join(data_dir, 'training_settings.yaml'), 'w') as fp:
                yaml.dump(training_settings, fp)
            logger.info('Training settings stored.')
            logger.info('Stored train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma))

[18:57:00.079] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[18:57:00.088] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[18:57:00.088] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=50|baselinerate=5|behaviour_sigma=0.5
[18:57:00.089] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[18:57:00.096] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[18:57:00.097] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=100|baselinerate=5|behaviour_sigma=0.5
[18:57:00.097] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[18:57:00.105] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[18:57:00.105] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=200|baselinerate=5|behaviour_sigma=0.5
[18:57:00.106] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[18:57:00.113] INFO [latentneural.utils.logging.<module>:84] Training se